In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

<b><font size="6" color="red">Ch.7 Attention (스마트번역기)</font><b>

# 1. 패키지 및 하이퍼파라미터

In [2]:
import numpy as np
import pandas as pd
from time import time
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼파라미터
MY_HIDDEN = 128
MY_EPOCH  = 500

# 2. 학습 데이터

In [3]:
raw = pd.read_csv('data/translate.csv', header=None)
eng_kor = raw.values.tolist() #데이터프레임을 list로 변환
print(eng_kor[:3])
print('영한 번역 데이터 개수:', len(eng_kor))

[['cold', '감기'], ['come', '오다'], ['cook', '요리']]
영한 번역 데이터 개수: 110


# 3. 영어알파벳과 한글문자 리스트 만들기

In [4]:
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
korean = ''.join([data[1] for data in eng_kor])
k_ch = list(set([c for c in korean]))
k_ch.sort()
k_alpha = pd.read_csv('data/korean.csv', header=None)[0].tolist()
k_ch == k_alpha # 순서와 내용이 같은지

True

In [6]:
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳:', alpha)
alpha_total_size =len(alpha) #171 원핫 인코딩 사이즈
print('전체 알파벳 개수(원핫인코딩 사이즈):', alpha_total_size)

영어와 한글 알파벳: ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 개수(원핫인코딩 사이즈): 171


# 4. 문자당 num(id)를 갖는 dictionary

In [7]:
# char_to_num = {}
# for i, c in enumerate(alpha):
#     char_to_num[c] = i
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [31]:
# 원핫인코딩 방법 2 - to_categorical([5,3,7], num_classes=10)
to_categorical([5,3,7], num_classes=10)

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

# 5. 인코더 입력, 디코더 입력, 디코더 출력
- 인코더 입력 디코더 입력은 원핫인코딩O
- 디코더 출력은 원핫인코딩X

In [8]:
def encoding(eng_kor=eng_kor):
    '인코더 입력, 디코더 입력, 디코더 출력 데이터를 return'
    enc_in = [] # 인코더 입력
    dec_in = [] # 디코더 입력
    dec_out = [] # 디코더 출력
    for data in eng_kor:
        # 인코더 입력 데이터(영어 → 숫자 → 원핫인코딩)
        eng = [char_to_num[ch] for ch in data[0]]
        # eng_one = to_categorical(eng, num_classes=alpha_total_size)
        eng_one = np.eye(alpha_total_size)[eng]
        enc_in.append(eng_one)
        # 디코더 입력 데이터('S한글' → 숫자 → 원핫인코딩)
        kor = [char_to_num[ch] for ch in 'S'+data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        # print(kor, kor_one)
        dec_in.append(kor_one)
        # 디코더 출력 ('한글E' → 숫자)
        kor = [char_to_num[ch] for ch in data[1]+'E']
        dec_out.append(kor)
    # 인공신경망에 넣을 데이터이므로 numpy array로 전환
    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    dec_out = np.array(dec_out)
    return enc_in, dec_in, dec_out
sample = [['wood', '나무'], ['word', '단어']]

# 6. 전체 번역 데이터(독립변수와 타깃변수)

In [9]:
X_enc, X_dec, y_dec = encoding(eng_kor)
Y_dec = np.expand_dims(y_dec, axis=-1)
X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

# 7-1. Seq2seq 모델 구현 (ch6)

In [48]:
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))
# 윗출력, state_c (Cell State), state_h (Hidden State)
_, state_h, state_c = LSTM(units=MY_HIDDEN, 
                           return_state=True
                           # return_sequences=False # 윗출력 안 받음
                          )(ENC_IN)
# 인코더와 디코더를 연결할 link
link = [state_h, state_c]

# 디코더 구현 : return_sequences = True 위로 올라가는 출력값 사용
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN, 
               return_sequences=True, 
               #return_state=False 옆으로 보내는 값 없음
              )(DEC_IN, initial_state=link)

# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size, 
                activation='softmax'
               )(DEC_MID)

# 모델
model = Model(inputs=[ENC_IN, DEC_IN], 
              outputs=DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 128),        153600      ['input_3[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

# 7-2. Attention 모델 구현

* 이번 예제에선 크게 의미 없어. 다만, 챗봇 등에선 효과 있지
* Attention 내부에 softmax 활성화함수 내장 - 인코더의 어느 단어에 '집중'할 것인가?
    - 유사도 점수(score) 입력 - 어텐션 가중치(Weights) 출력
* 최종 출력 전 Dense의 softmax - 다음에 올 글자가 무엇인가?
    - 컨텍스트 벡터 + 디코더 정보 입력 - 최종 단어의 확률분포 출력

In [10]:
from tensorflow.keras.layers import Attention, Concatenate
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))
# 윗출력, state_c (Cell State), state_h (Hidden State)
ENC_OUT, state_h, state_c = LSTM(units=MY_HIDDEN, 
                           return_state=True,
                           return_sequences=True # 윗출력 받음
                          )(ENC_IN)
# 인코더와 디코더를 연결할 link
link = [state_h, state_c]

# 디코더 구현 : return_sequences = True 위로 올라가는 출력값 사용
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN, 
               return_sequences=True, 
               #return_state=False 옆으로 보내는 값 없음
              )(DEC_IN, initial_state=link)

# 어텐션 메커니즘 (1~3)
CONTEXT_VECTOR = Attention()([DEC_MID, ENC_OUT])

# CONTEXT_VECTOR와 DECODER LSTM 출력을 결합 (4)
CONTEXT_AND_LSTM_OUT = Concatenate()([CONTEXT_VECTOR, DEC_MID])

# 최종 출력층
OUT = Dense(units=alpha_total_size, 
            activation='softmax'
           )(CONTEXT_AND_LSTM_OUT)

# 모델
model = Model(inputs=[ENC_IN, DEC_IN], 
              outputs=OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 4, 128),     153600      ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

# 8. 학습과정 설정 및 학습하기

In [11]:
model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='rmsprop', 
    metrics=['accuracy']
)
start = time()
# from tensorflow.keras.callbacks import EarlyStopping
# earlyStopping = EarlyStopping(monitor='accuracy', patience=50)
hist = model.fit(
    [X_enc, X_dec], Y_dec, 
    epochs=MY_EPOCH,
    verbose=1
)
end = time()
print(f'학습시간: {end-start}')

Epoch 1/500
4/4 [==============================] - 2s 8ms/step - loss: 5.1126 - accuracy: 0.2212
Epoch 2/500
4/4 [==============================] - 0s 7ms/step - loss: 4.9495 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 7ms/step - loss: 4.2376 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 9ms/step - loss: 3.4583 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 5ms/step - loss: 3.3997 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 6ms/step - loss: 3.3583 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 6ms/step - loss: 3.3233 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 7ms/step - loss: 3.2945 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 10ms/step - loss: 3.2663 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 6ms/step - loss: 3.2419 - accuracy: 0.3333
Epoch 11/500
4/4 [==========

4/4 [==============================] - 0s 7ms/step - loss: 0.5658 - accuracy: 0.9455
Epoch 85/500
4/4 [==============================] - 0s 6ms/step - loss: 0.6010 - accuracy: 0.9333
Epoch 86/500
4/4 [==============================] - 0s 5ms/step - loss: 0.5209 - accuracy: 0.9636
Epoch 87/500
4/4 [==============================] - 0s 6ms/step - loss: 0.4985 - accuracy: 0.9515
Epoch 88/500
4/4 [==============================] - 0s 7ms/step - loss: 0.4783 - accuracy: 0.9636
Epoch 89/500
4/4 [==============================] - 0s 10ms/step - loss: 0.4589 - accuracy: 0.9697
Epoch 90/500
4/4 [==============================] - 0s 6ms/step - loss: 0.4453 - accuracy: 0.9697
Epoch 91/500
4/4 [==============================] - 0s 5ms/step - loss: 0.4206 - accuracy: 0.9758
Epoch 92/500
4/4 [==============================] - 0s 6ms/step - loss: 0.4073 - accuracy: 0.9758
Epoch 93/500
4/4 [==============================] - 0s 7ms/step - loss: 0.3835 - accuracy: 0.9697
Epoch 94/500
4/4 [==============

4/4 [==============================] - 0s 12ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 167/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 168/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0115 - accuracy: 1.0000
Epoch 169/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 170/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 171/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 172/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0086 - accuracy: 1.0000
Epoch 173/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0126 - accuracy: 0.9970
Epoch 174/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 175/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0061 - accuracy: 1.0000
Epoch 176/500
4/4 [===

4/4 [==============================] - 0s 7ms/step - loss: 7.8224e-05 - accuracy: 1.0000
Epoch 248/500
4/4 [==============================] - 0s 10ms/step - loss: 1.0963e-04 - accuracy: 1.0000
Epoch 249/500
4/4 [==============================] - 0s 7ms/step - loss: 6.3683e-05 - accuracy: 1.0000
Epoch 250/500
4/4 [==============================] - 0s 7ms/step - loss: 5.6957e-05 - accuracy: 1.0000
Epoch 251/500
4/4 [==============================] - 0s 6ms/step - loss: 5.3231e-05 - accuracy: 1.0000
Epoch 252/500
4/4 [==============================] - 0s 7ms/step - loss: 5.0442e-05 - accuracy: 1.0000
Epoch 253/500
4/4 [==============================] - 0s 7ms/step - loss: 4.9225e-05 - accuracy: 1.0000
Epoch 254/500
4/4 [==============================] - 0s 6ms/step - loss: 4.6790e-05 - accuracy: 1.0000
Epoch 255/500
4/4 [==============================] - 0s 6ms/step - loss: 4.4782e-05 - accuracy: 1.0000
Epoch 256/500
4/4 [==============================] - 0s 6ms/step - loss: 5.2566e-05 - 

4/4 [==============================] - 0s 7ms/step - loss: 2.4907e-06 - accuracy: 1.0000
Epoch 327/500
4/4 [==============================] - 0s 9ms/step - loss: 2.3939e-06 - accuracy: 1.0000
Epoch 328/500
4/4 [==============================] - 0s 6ms/step - loss: 2.4232e-06 - accuracy: 1.0000
Epoch 329/500
4/4 [==============================] - 0s 6ms/step - loss: 2.2621e-06 - accuracy: 1.0000
Epoch 330/500
4/4 [==============================] - 0s 7ms/step - loss: 2.1877e-06 - accuracy: 1.0000
Epoch 331/500
4/4 [==============================] - 0s 9ms/step - loss: 2.1981e-06 - accuracy: 1.0000
Epoch 332/500
4/4 [==============================] - 0s 5ms/step - loss: 2.1194e-06 - accuracy: 1.0000
Epoch 333/500
4/4 [==============================] - 0s 6ms/step - loss: 2.0934e-06 - accuracy: 1.0000
Epoch 334/500
4/4 [==============================] - 0s 6ms/step - loss: 2.0143e-06 - accuracy: 1.0000
Epoch 335/500
4/4 [==============================] - 0s 6ms/step - loss: 1.9218e-06 - a

4/4 [==============================] - 0s 6ms/step - loss: 6.7805e-07 - accuracy: 1.0000
Epoch 406/500
4/4 [==============================] - 0s 7ms/step - loss: 6.6902e-07 - accuracy: 1.0000
Epoch 407/500
4/4 [==============================] - 0s 9ms/step - loss: 6.6251e-07 - accuracy: 1.0000
Epoch 408/500
4/4 [==============================] - 0s 5ms/step - loss: 6.5601e-07 - accuracy: 1.0000
Epoch 409/500
4/4 [==============================] - 0s 6ms/step - loss: 6.5782e-07 - accuracy: 1.0000
Epoch 410/500
4/4 [==============================] - 0s 7ms/step - loss: 6.5565e-07 - accuracy: 1.0000
Epoch 411/500
4/4 [==============================] - 0s 5ms/step - loss: 6.4192e-07 - accuracy: 1.0000
Epoch 412/500
4/4 [==============================] - 0s 7ms/step - loss: 6.3795e-07 - accuracy: 1.0000
Epoch 413/500
4/4 [==============================] - 0s 7ms/step - loss: 6.2603e-07 - accuracy: 1.0000
Epoch 414/500
4/4 [==============================] - 0s 6ms/step - loss: 6.3217e-07 - a

4/4 [==============================] - 0s 7ms/step - loss: 3.7713e-07 - accuracy: 1.0000
Epoch 485/500
4/4 [==============================] - 0s 9ms/step - loss: 3.7424e-07 - accuracy: 1.0000
Epoch 486/500
4/4 [==============================] - 0s 6ms/step - loss: 3.7208e-07 - accuracy: 1.0000
Epoch 487/500
4/4 [==============================] - 0s 6ms/step - loss: 3.7316e-07 - accuracy: 1.0000
Epoch 488/500
4/4 [==============================] - 0s 6ms/step - loss: 3.6594e-07 - accuracy: 1.0000
Epoch 489/500
4/4 [==============================] - 0s 5ms/step - loss: 3.6594e-07 - accuracy: 1.0000
Epoch 490/500
4/4 [==============================] - 0s 6ms/step - loss: 3.6341e-07 - accuracy: 1.0000
Epoch 491/500
4/4 [==============================] - 0s 7ms/step - loss: 3.6305e-07 - accuracy: 1.0000
Epoch 492/500
4/4 [==============================] - 0s 9ms/step - loss: 3.6124e-07 - accuracy: 1.0000
Epoch 493/500
4/4 [==============================] - 0s 6ms/step - loss: 3.5907e-07 - a

In [ ]:
eng_kor

In [ ]:
# 쉬운 문제
easy_test = [
    ['find','PP'], 
    ['cold', 'PP'], 
    ['desk', 'PP'], 
    ['soon', 'PP'], 
    ['love', 'PP']
]
enc_in, dec_in, dec_out = encoding(easy_test)
enc_in.shape, dec_in.shape

In [ ]:
pred = model.predict([enc_in, dec_in]) #변수는 하나여야 하니 리스트로 넣는다
pred.shape

In [ ]:
for i in range(len(pred)):
    eng = easy_test[i][0]
    hat = pred[i].argmax(axis=-1)[:-1]
#    kor = alpha[hat[0]] + alpha[hat[0]]
    kor = ''.join([alpha[num] for num in hat])
    print("{} → {} {}".format(eng, kor, hat))

In [ ]:
# 어려운 문제 (철자 순서 )
hard_test = [
    ['love', 'PP'],
    ['lveo', 'PP'],
    ['leov', 'PP'],
    ['levo', 'PP'],
    ['lvoe', 'PP'],
    ['loev', 'PP'],
    ['elov', 'PP'],
    ['evol', 'PP'],
    ['lobe', 'PP']
]
enc_in, dec_in, _ = encoding(hard_test)
enc_in.shape, dec_in.shape

In [ ]:
# # 어려운 문제 (철자 순서 )
# hard_test = [
#     ['fnid', 'PP'],
#     ['codl', 'PP'],
#     ['dske', 'PP'],
#     ['noos', 'PP'],
#     ['lvoe', 'PP']
# ]
# enc_in, dec_in, _ = encoding(hard_test)
# enc_in.shape, dec_in.shape

In [ ]:
pred = model.predict([enc_in, dec_in], verbose=0).argmax(axis=-1)
pred

In [ ]:
for i in range(len(pred)):
    eng = hard_test[i][0]
    kor = ''.join([alpha[num] for num in pred[i]])
    print("{} → {} {}".format(eng, kor[:-1], pred[i][:-1]))